## Code and Documentation to translate Popup Buoy Transmitted/Recorded Data

## Data Structure

Data downloaded from Popup Buoy's directly, generate 8 binary files.  A complete description of these files can be found at **[link]**.  Relevant pieces will be included in the notebook.

Filenames:
- BOTDAT.TXT
- FILEPOS.TXT
- ICEDAT.TXT
- JPGxxxxx.JPG
- PRODAT.TXT
- SSTDAT.TXT
- SUMMARY.TXT

In [1]:
input_file = 'sampledata/BOTDAT.TXT'

In [4]:
def HexView(file):
    with open(file, 'rb') as in_file:
        while True:
            hexdata = in_file.read(17).hex().upper()     # Read the shortest possible line
            if hexdata[0:4] != 'FFFF':
                #in_file.seek(-16,1)
                print('help',hexdata[0:4])
                hexdata = in_file.read(17).hex().upper() 
            if len(hexdata) == 0:                # breaks loop once no more binary data is read
                break
            print(hexdata.upper())               # I also like it all in caps. 
            
    return(hexdata.upper())

In [5]:
hexstr = HexView(input_file)

FFFF000049BE298E2960214E1D30005907
FFFF000149C0296C2944214F1A2C004A07
FFFF000249C029502933214F18AC003D07
FFFF000349C229452931214F17A0003D07
FFFF000449C3294A293B214F15F0004507
FFFF000549C429542949214F1558004307
FFFF000649C629612957214F1552004207
FFFF000749C529682962214F1552003407
FFFF000849C72975296F214F1552003707
FFFF000949C929842980214F1552003607
FFFF000A49C92993298F214F1552002F07
FFFF000B49CA29A329A0214F1552004C07
FFFF000C49CA29B329B0214F1552003D07
FFFF000D49CA29BE29BD214F1552003807
FFFF000E49CB29CC29CB214F1552002E07
FFFF000F49CC29D729D5214F1552003E07
FFFF001049CD29E329E2214F15C0003A07
help 
